In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers # type: ignore
from tensorflow.keras.layers import Dense, Input # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data() # 학습셋과 평가셋 분리해서 로드
is_normalized = False

In [4]:
if not is_normalized: # is_normalized 변수가 False라면
  train_images = train_images / 255.0 # 0~255 to 0~1
  test_images = test_images / 255.0 # 0~255 to 0~1

  is_normalized = True # 정규화 완료

In [ ]:
import tensorflow as tf
from tensorflow import keras

# copy from https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e

from IPython.display import clear_output
from tensorflow.keras.callbacks import Callback

class PlotLosses(Callback):

  def on_train_begin(self, logs={}): # 훈련이 시작될 때 케라스가 자동으로 호출해주는 핵심 구간

    self.i = 0
    self.x = []
    self.losses = []
    self.val_losses = []

    self.fig = plt.figure()

    self.logs = []


  def on_epoch_end(self, epoch, logs={}): # 매 에포크가 끝날 때마다 케라스가 자동으로 호출해주는 핵심 구간

    self.logs.append(logs)
    self.x.append(self.i)
    self.losses.append(logs.get('loss'))
    self.val_losses.append(logs.get('val_loss'))
    self.i += 1

    clear_output(wait=True) # 이전 에포크에서 그렸던 그래프를 지워줍니다.
    plt.plot(self.x, self.losses, label="loss")
    plt.plot(self.x, self.val_losses, label="val_loss")
    plt.legend()
    plt.show()
    print("loss = ", self.losses[-1], ", val_loss = ", self.val_losses[-1])



# 입력층
inputs = keras.Input(shape=(28, 28))
x = keras.layers.Flatten()(inputs)

# 은닉층 (Dense 기반)
x = keras.layers.Dense(1024, activation='gelu', kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Dense(512, activation='gelu', kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.10)(x)

x = keras.layers.Dense(256, activation='gelu', kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.05)(x)

x = keras.layers.Dense(128, activation='gelu', kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)

# 출력층 (분류 문제 → Softmax)
outputs = keras.layers.Dense(10, activation='softmax')(x)

# 모델 정의
model = keras.Model(inputs=inputs, outputs=outputs)

# compile도 한 번만
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# -------------------------
# 콜백 정의
# -------------------------

model_check_point = ModelCheckpoint('best_model.keras', monitor='val_loss', mode='min', save_best_only=True)
plot_losses = PlotLosses()
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_learning_rate=0.001)
callbacks = [
    keras.callbacks.EarlyStopping(
        patience=8, restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=4,
        min_lr=1e-5
    )
]

# -------------------------
# 학습 (Functional DNN 모델에 맞게 수정)
# -------------------------
history = model.fit(
    train_images, 
    keras.utils.to_categorical(train_labels, 10),   # 원-핫 인코딩
    epochs=40,
    batch_size=64,
    validation_data=(
        test_images, 
        keras.utils.to_categorical(test_labels, 10) # 원-핫 인코딩
    ),
    callbacks=callbacks,
    verbose=1
)



Epoch 1/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - accuracy: 0.8324 - loss: 0.4647 - val_accuracy: 0.8519 - val_loss: 0.4214 - learning_rate: 0.0010
Epoch 2/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.8641 - loss: 0.3673 - val_accuracy: 0.8476 - val_loss: 0.4242 - learning_rate: 0.0010
Epoch 3/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.8735 - loss: 0.3427 - val_accuracy: 0.8737 - val_loss: 0.3493 - learning_rate: 0.0010
Epoch 4/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.8787 - loss: 0.3256 - val_accuracy: 0.8751 - val_loss: 0.3558 - learning_rate: 0.0010
Epoch 5/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.8837 - loss: 0.3110 - val_accuracy: 0.8795 - val_loss: 0.3386 - learning_rate: 0.0010
Epoch 6/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.8908 - loss: 0.2926 - val_accuracy: 0.8703 - val_loss: 0.3660 - learning_rate: 0.0010
Epoch 7/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - accuracy: 0.8946 - l